In [7]:

import pandas as pd
import time
import datetime
from datetime import date
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome import service
import pygame
pygame.init()

# master_match_id_list = list(range(1640674,1641053)) #match ids for the season

# Creating functions for events displayed as icons on the live statistics webpage. 
# Referenced in the summary steps of home and away functions
def replace_pd(df):
    # mapping pattern; replace hyphens with 0s in whoscored data
    mapping = {'-': 0}
    replace_dict = {}
    for column in df.columns:
        replace_dict[column] = mapping
    return df.replace(replace_dict)

def goal_check(element):
    # find goal scored and penalty scored events
    goal = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-goalnormal")
    goal = goal + element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-penaltyscored")
    return goal

def assist_check(element):
    # find event string for assist
    assist = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-intentionalassist")
    #This might need adjustments as more assists occur. theres like 3+ strings for an assist
    return assist

def owngoal_check(element):
    # find event string for own goal
    owngoal = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-goalown")
    return owngoal

def yellow_check(element):
    # find event string for yellow card
    yellow = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-yellowcard")
    return yellow

def red_check(element):
    # find event string for red card
    red = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-redcard")
    return red

def lmt_check(element):
    lmt = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-tacklelastman")
    return lmt

def glc_check(element):
    glc = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-clearanceofftheline")
    return glc

def elg_check(element):
    elg = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-errorleadstogoal")
    return elg

def pksave_check(element):
    pksave = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-keeperpenaltysaved")
    return pksave

def motm_check(element):
    motm = element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").count("data-mom")
    return motm

def get_suboffmin(element):
    if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").find("data-event-satisfier-suboff") != -1:
        if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").count("data-mom") == 1:
            suboffmin = element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"span")[4].get_attribute("innerHTML")
        else: 
            suboffmin = element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"span")[3].get_attribute("innerHTML")
    else:
        suboffmin = 0
    if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").find("data-event-satisfier-suboff") != -1:
        if len(suboffmin) != 5:
            if len(suboffmin) == 4:
                suboffmin = suboffmin[1]
            else:
                suboffmin = suboffmin[1]+suboffmin[2]+suboffmin[3]
        else:
            suboffmin = suboffmin[1]+suboffmin[2]
    else:
        suboffmin = 0
    return suboffmin

def get_subonmin(element):
    if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").find("data-event-satisfier-subon") != -1:
        if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").count("data-mom") == 1:
            subonmin = element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"span")[4].get_attribute("innerHTML")
        else:
            subonmin = element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"span")[3].get_attribute("innerHTML")            
    else:
        subonmin = 0
    if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").find("data-event-satisfier-subon") != -1:
        if len(subonmin) != 5:
            if len(subonmin) == 4:
                subonmin = subonmin[1]
            else:
                subonmin = subonmin[1]+subonmin[2]+subonmin[3]
        else:
            subonmin = subonmin[1]+subonmin[2]
    else:
        subonmin = 0
    return subonmin

def home_players(match_id, api_delay_term=5):

# connect webdriver
    url = "https://www.whoscored.com/Matches/" + str(match_id) + "/LiveStatistics/"
    driver =  webdriver.Chrome('./chromedriver')
    driver.get(url)
    
    #
    #chop = webdriver.ChromeOptions()
    #chop.add_extension('Adblock-Plus_v1.4.1.crx')
    #driver = webdriver.Chrome(chrome_options = chop)
    #
    
#     close pop up ad
    try:
        driver.find_element(By.TAG_NAME,'button').click()
    except:
        pass
    
    
# make pandas summary dataframe
        # add in data type filters for events. 
        #Goal: 16 data-event-satisfier-goalnormal
        #Assist: 1 data-event-satisfier-assist
        #pksave data-event-satisfier-keeperpenaltysaved
        #Yellow: 17, card type 31 data-event-satisfier-yellowcard
        #Red: 17, 33 data-event-satisfier-redcard
        #Subbed On: 19 data-event-satisfier-subon
        #Subbed Off: 18 data-event-satisfier-suboff
        #Goal Line Clearance: 10 data-event-satisfier-clearanceofftheline
        #Last Man Tackle 7 data-event-satisfier-tacklelastman
        #Penalty Saved: 58 data-event-satisfier-keeperpenaltysaved
        #Error Lead to Goal: 51 data-event-satisfier-errorleadstogoal
        #Own Goal: 16 data-event-satisfier-goalown
        #Man of the Match: data-mom
        #functions created in lines above
    player_summary_df = pd.DataFrame(columns=[
            "matchid", "player", "date", "team", "accessed", "opponent", "Loc", "G", "A", "PKSave", "Ylw", "Red", "GLC", "LMT", "ELG", "OG", "MotM", "SubOffMin", "SubOnMin"
        ])        

# get player summary datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-summary #player-table-statistics-body tr")
    for element in elements:

        player_sum_dict = {
            "matchid": match_id,
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "date": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[2]/span[3]/div[3]/dl/dd[2]").text.split(', ')[1],
            "team": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[1]/span[1]/a").text,
            "accessed": date.today(),
            "opponent": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[1]/span[5]/a").text,
            "Loc": "Home",
            "G": goal_check(element),   
            "A": assist_check(element),
            "PKSave": pksave_check(element),
            "Ylw": yellow_check(element),
            "Red": red_check(element),
            "GLC": glc_check(element),
            "LMT": lmt_check(element),
            "ELG": elg_check(element),
            "OG": owngoal_check(element),
            "MotM": motm_check(element),
            "SubOffMin": get_suboffmin(element),
            "SubOnMin": get_subonmin(element),
        }

        player_summary_df.loc[len(player_summary_df)] = player_sum_dict
        
# make pandas offense dataframe
    player_offensive_df = pd.DataFrame(columns=[
            "player", "SoT", "KP", "Drb"
        ])

    # wait for getting data
    time.sleep(api_delay_term)

    # click event for getting offensive data
    driver.find_element(By.CSS_SELECTOR,"#live-player-home-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[1]/div[1]/ul/li[2]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player offensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-offensive #player-table-statistics-body tr")
    for element in elements:
       
        player_off_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "SoT": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "KP": element.find_elements(By.CSS_SELECTOR,"td")[4].text, 
            "Drb": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_offensive_df.loc[len(player_offensive_df)] = player_off_dict

# make pandas defense dataframe
    player_defensive_df = pd.DataFrame(columns=[
            "player", "tkl", "int", "clr", "blk"
        ])    
    
    # wait for getting data
    time.sleep(api_delay_term)
    
    # click event for getting defensive data
    driver.find_element(By.CSS_SELECTOR,"#live-player-home-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[1]/div[1]/ul/li[3]/a").click()
    
    # wait for getting data
    time.sleep(api_delay_term)
    
# get player defensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-defensive #player-table-statistics-body tr")
    for element in elements:
       
        player_def_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "tkl": element.find_elements(By.CSS_SELECTOR,"td")[2].text, 
            "int": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "clr": element.find_elements(By.CSS_SELECTOR,"td")[4].text,
            "blk": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_defensive_df.loc[len(player_defensive_df)] = player_def_dict

# make pandas passing dataframe
    player_passing_df = pd.DataFrame(columns=[
            "player", "AccCross", "AccLB"
        ])

    # wait for getting data
    time.sleep(api_delay_term)

    # click event for getting passing data
    driver.find_element(By.CSS_SELECTOR,"#live-player-home-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[1]/div[1]/ul/li[4]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player passing datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-passing #player-table-statistics-body tr")
    for element in elements:
       
        player_pass_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "AccCross": element.find_elements(By.CSS_SELECTOR,"td")[6].text, 
            "AccLB": element.find_elements(By.CSS_SELECTOR,"td")[8].text, 
        }
        
        player_passing_df.loc[len(player_passing_df)] = player_pass_dict

# Get Saves

    url = "https://www.whoscored.com/Matches/" + str(match_id) + "/Live"
    driver.get(url)

    driver.find_element(By.XPATH,"/html/body/div[4]/div[4]/ul/li[3]/a").click()
    time.sleep(api_delay_term)
    driver.find_element(By.XPATH,"/html/body/div[4]/div[5]/div[4]/ul/li[14]/a").click()
    time.sleep(api_delay_term)

    player_saves_df = pd.DataFrame(columns=[
            "player", "saves"
        ])        
    elements = driver.find_elements(By.XPATH,"/html/body/div[4]/div[5]/div[4]/div[1]/div[2]/ul/div")
    for element in elements:
        
        player_saves_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"div")[0].find_elements(By.CSS_SELECTOR,"span")[1].get_attribute("title"),
            "saves": element.find_elements(By.CSS_SELECTOR,"div")[1].get_attribute("innerText").split('\n')[0]
        }
        
        player_saves_df.loc[len(player_saves_df)] = player_saves_dict

        
# close webdriver
    driver.close()
    driver.quit()

#merge data frames
    player_so_df = pd.merge(player_summary_df,player_offensive_df,on="player")
    player_sod_df = pd.merge(player_so_df,player_defensive_df,on="player")
    player_sods_df = pd.merge(player_sod_df,player_saves_df,on="player")
    player_all_df = pd.merge(player_sods_df,player_passing_df,on="player")
    
    # New CSV, one-off header population
#     player_all_df.to_csv('wsdata.csv')
    
    # Append data to existing csv
    player_all_df.to_csv('wsdata_23_24.csv',mode='a',header=False)
    
    return replace_pd(player_all_df)

def away_players(match_id, api_delay_term=5):

# connect webdriver
    url = "https://www.whoscored.com/Matches/" + str(match_id) + "/LiveStatistics/"
    driver =  webdriver.Chrome('./chromedriver')
    driver.get(url)
    
    try:
        driver.find_element(By.TAG_NAME,'button').click()
    except:
        pass

    
# make pandas summary dataframe
    player_summary_df = pd.DataFrame(columns=[
            "matchid", "player", "date", "team", "accessed", "opponent", "Loc", "G", "A", "PKSave", "Ylw", "Red", "GLC", "LMT", "ELG", "OG", "MotM", "SubOffMin", "SubOnMin"
        ])        
    
# get player summary datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-summary #player-table-statistics-body tr")
    for element in elements:

        player_sum_dict = {
            "matchid": match_id,
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "date": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[2]/span[3]/div[3]/dl/dd[2]").text.split(', ')[1],
            "team": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[1]/span[5]/a").text,
            "accessed": date.today(),
            "opponent": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[1]/span[1]/a").text,
            "Loc": "Away",
            "G": goal_check(element),   
            "A": assist_check(element),
            "PKSave": pksave_check(element),
            "Ylw": yellow_check(element),
            "Red": red_check(element),
            "GLC": glc_check(element),
            "LMT": lmt_check(element),
            "ELG": elg_check(element),
            "OG": owngoal_check(element),
            "MotM": motm_check(element),
            "SubOffMin": get_suboffmin(element),
            "SubOnMin": get_subonmin(element),
        }

        player_summary_df.loc[len(player_summary_df)] = player_sum_dict
        
# make pandas offense dataframe
    player_offensive_df = pd.DataFrame(columns=[
        "player", "SoT", "KP", "Drb"
    ])

    # wait for getting data
    ActionChains(driver).scroll_by_amount(delta_x=0,delta_y=1500).perform()
    time.sleep(api_delay_term)

    # click event for getting offensive data
    driver.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[3]/div[1]/ul/li[2]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player offensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-offensive #player-table-statistics-body tr")
    for element in elements:
       
        player_off_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "SoT": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "KP": element.find_elements(By.CSS_SELECTOR,"td")[4].text, 
            "Drb": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_offensive_df.loc[len(player_offensive_df)] = player_off_dict


# make pandas defense dataframe
    player_defensive_df = pd.DataFrame(columns=[
            "player", "tkl", "int", "clr", "blk"
        ])    
    
    # wait for getting data
    ActionChains(driver).scroll_by_amount(delta_x=0,delta_y=1).perform()
    time.sleep(api_delay_term)
    
    # click event for getting defensive data
    driver.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[3]/div[1]/ul/li[3]/a").click()
    
    # wait for getting data
    time.sleep(api_delay_term)
    
# get player defensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-defensive #player-table-statistics-body tr")
    for element in elements:
       
        player_def_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "tkl": element.find_elements(By.CSS_SELECTOR,"td")[2].text, 
            "int": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "clr": element.find_elements(By.CSS_SELECTOR,"td")[4].text,
            "blk": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_defensive_df.loc[len(player_defensive_df)] = player_def_dict

        
# make pandas passing dataframe
    player_passing_df = pd.DataFrame(columns=[
            "player", "AccCross", "AccLB"
        ])

    # wait for getting data
    ActionChains(driver).scroll_by_amount(delta_x=0,delta_y=1).perform()
    time.sleep(api_delay_term)

    # click event for getting passing data
    driver.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[3]/div[1]/ul/li[4]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player passing datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-passing #player-table-statistics-body tr")
    for element in elements:
       
        player_pass_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "AccCross": element.find_elements(By.CSS_SELECTOR,"td")[6].text, 
            "AccLB": element.find_elements(By.CSS_SELECTOR,"td")[8].text, 
        }
        
        player_passing_df.loc[len(player_passing_df)] = player_pass_dict

# Get Saves

    url = "https://www.whoscored.com/Matches/" + str(match_id) + "/Live"
    driver.get(url)

    driver.find_element(By.XPATH,"/html/body/div[4]/div[4]/ul/li[3]/a").click()
    time.sleep(api_delay_term)
    driver.find_element(By.XPATH,"/html/body/div[4]/div[5]/div[4]/ul/li[14]/a").click()
    time.sleep(api_delay_term)

    player_saves_df = pd.DataFrame(columns=[
            "player", "saves"
        ])        
    elements = driver.find_elements(By.XPATH,"/html/body/div[4]/div[5]/div[4]/div[1]/div[4]/ul/div")
    for element in elements:
        
        player_saves_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"div")[0].find_elements(By.CSS_SELECTOR,"span")[1].get_attribute("title"),
            "saves": element.find_elements(By.CSS_SELECTOR,"div")[1].get_attribute("innerText").split('\n')[0]
        }
        
        player_saves_df.loc[len(player_saves_df)] = player_saves_dict
 
        
# close webdriver
    driver.close()
    driver.quit()

#merge data frames
    
    player_so_df = pd.merge(player_summary_df,player_offensive_df,on="player")
    player_sod_df = pd.merge(player_so_df,player_defensive_df,on="player")
    player_sods_df = pd.merge(player_sod_df,player_saves_df,on="player")
    player_all_df = pd.merge(player_sods_df,player_passing_df,on="player")
    
    # New CSV, one-off header population
    #player_all_df.to_csv('wsdata.csv')
    
    # Append data to existing csv
    player_all_df.to_csv('wsdata_23_24.csv',mode='a',header=False)
    
    return replace_pd(player_all_df)



In [8]:
#match_ids = list(range(1640872,1640873))
match_ids = [1729191]
#match_ids.sort()
#del match_ids[0]
# match_ids


In [9]:

for ids in match_ids:
    away_players(ids,api_delay_term=4)
    home_players(ids,api_delay_term=4)

# for ids in match_ids:
#     try:
#         away_players(ids,api_delay_term=4)
#     except:
#         pass

# for ids in match_list_temp:
#     try:
#         home_players(ids,api_delay_term=4)
#     except:
#         pass

pygame.mixer.Sound('./blob.mp3').play()


/var/folders/2k/vcc_jzmx2y9g1v6djwx1lb0h0000gn/T/ipykernel_8818/3072696621.py:309: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver =  webdriver.Chrome('./chromedriver')
/var/folders/2k/vcc_jzmx2y9g1v6djwx1lb0h0000gn/T/ipykernel_8818/3072696621.py:118: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver =  webdriver.Chrome('./chromedriver')


In [13]:

# Test new html attribute

# url = "https://www.whoscored.com/Matches/1640957/Live/"
# driver =  webdriver.Chrome('./chromedriver')
# driver.get(url)

# driver.find_element(By.XPATH,"/html/body/div[4]/div[4]/ul/li[3]/a").click()
# time.sleep(4)
# driver.find_element(By.XPATH,"/html/body/div[4]/div[5]/div[4]/ul/li[14]/a").click()
# time.sleep(4)

# saves = driver.find_elements(By.XPATH,"/html/body/div[4]/div[5]/div[4]/div[1]/div[4]/ul/div")[0].find_elements(By.CSS_SELECTOR,"div")[1].get_attribute("innerText").split('\n')[0]
# # find_elements(By.CSS_SELECTOR,"div")[0].find_elements(By.CSS_SELECTOR,"span")[1].get_attribute("title"),
# # element.find_elements(By.CSS_SELECTOR,"div")[1].get_attribute("innerText").split('\n')[0]

# driver.close()
# driver.quit()

# saves

/var/folders/2k/vcc_jzmx2y9g1v6djwx1lb0h0000gn/T/ipykernel_44914/1413994188.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver =  webdriver.Chrome('./chromedriver')


'5'

In [3]:
# entries = pd.read_csv('wsdata.csv')
# entries

In [4]:
# teams = ['Burnley','Luton','Sheffield United']
# new_players = ['Jurriën Timber','Pau Torres','Nicolas Jackson','Arnaut Danjuma','Romain Faivre','Hamed Traorè','Sandro Tonali','Mark Flekken','Kevin Schade','Mahmoud Dahoud','Jordan Beyer','Christopher Nkunku','Dominik Szoboszlai','João Pedro','Matheus Cunha','Matt Doherty','Moussa Diaby',"Dara O'Shea",'Michael Obafemi','Tahith Chong','Milos Kerkez','Yasser Larouci','Ethan Ampadu','Mads Andersen','Igor Julio','Nathan Redmond','Ola Aina','Jacob Bruun Larsen','Axel Disasi','Calvin Bassey','Rasmus Højlund','Lesley Ugochukwu','Josko Gvardiol']

# entries_champ = entries[entries['team'].isin(teams)]
# entries_champ

# entries_foreign = entries[entries['player'].isin(new_players)]
# entries_foreign

# new_entries = entries_champ.append(entries_foreign,ignore_index=True)
# new_entries = new_entries.drop_duplicates()


/var/folders/2k/vcc_jzmx2y9g1v6djwx1lb0h0000gn/T/ipykernel_21919/1278641452.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_entries = entries_champ.append(entries_foreign,ignore_index=True)


In [5]:
# new_entries.to_csv('wsdata_22_23_adds.csv',index=False)

In [6]:
# tableau1 = pd.read_csv('wsdata_2022_2023.csv')
# tableau2 = pd.read_csv('wsdata_22_23_adds.csv')
# tableau2 = tableau2.drop(columns = ['Loc','opponent'])
# # tableau2

# tableau_wsdata_22_23 = tableau1.append(tableau2,ignore_index=True)
# tableau_wsdata_22_33 = tableau_wsdata_22_23.drop_duplicates()
# tableau_wsdata_22_23.to_csv('tableau_wsdata_22_23.csv',index=False)

/var/folders/2k/vcc_jzmx2y9g1v6djwx1lb0h0000gn/T/ipykernel_21919/2703198750.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tableau_wsdata_22_23 = tableau1.append(tableau2,ignore_index=True)
